In [ ]:
import pickle
import numpy as np
import os
import mne
import pandas as pd
pickle.format_version

In [ ]:
main_path = os.getcwd()
all_file = os.listdir()
file_name = list()
for file in all_file:
    if file[0:3] == 'exp':
        file_name.append(file)
file_name = sorted(file_name)
file = file_name[8:] + file_name[0:8]
print(file)

In [ ]:
df1 = pd.read_csv(main_path + '/' + file[0])
df1 = df1.drop(columns=['Unnamed: 0', '0', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'])
df1.columns = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
df1

In [ ]:
# Drop some columns and save it into the new file.
for i in range(len(file)):
    converted_signal_path = main_path + '/Converted_signal/' + 'Conv_' + file[i]
    df = pd.read_csv(main_path + '/' + file[i])
    df = df.drop(columns=['Unnamed: 0', '0', '17', '18', '19', '20', '21', '22',
        '23', '24', '25', '26', '27', '28', '29', '30', '31'])
    df.columns = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
    df = df.set_index(df.columns[0])
    df.to_csv(converted_signal_path)

In [ ]:
def df_to_raw(df):
    ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
    ch_types = ['eeg'] * len(ch_names)
    df = df.T
    sampling_rate = 250 #Hz
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sampling_rate)
    raw = mne.io.RawArray(df, info)
    raw.set_montage('standard_1020')
    return raw

In [ ]:
from mne import Epochs, find_events

def getEpochs(raw, event_id, tmin, tmax, picks):

    #epoching
    events = find_events(raw)
    epochs = Epochs(raw, events=events, event_id=event_id, 
                    tmin=tmin, tmax=tmax, baseline=None, preload=True,verbose=False, picks=picks)  #8 channels
    print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)

    return epochs

In [2]:
# Get the name of converted data

main_path = os.getcwd()
all_file = os.listdir(main_path + '/Converted_signal')
conv_list = all_file[8:] + all_file[0:8]
conv_list

['Conv_exp1_Ati.csv',
 'Conv_exp2_Tonson.csv',
 'Conv_exp3_Pyae.csv',
 'Conv_exp4_Duc.csv',
 'Conv_exp5_And.csv',
 'Conv_exp6_Sayaka.csv',
 'Conv_exp7_Suyoga.csv',
 'Conv_exp8_Arti.csv',
 'Conv_exp9_Amanda.csv',
 'Conv_exp10_Sapna.csv',
 'Conv_exp11_Data.csv',
 'Conv_exp12_Jirasak.csv',
 'Conv_exp13_Max.csv',
 'Conv_exp14_Ayush.csv',
 'Conv_exp15_Dipesh.csv',
 'Conv_exp16_Shashank.csv',
 'Conv_exp17_Mi.csv']

In [3]:
file_path = main_path + '/Converted_signal/'
all_data = []
for file in conv_list:
    path = file_path + file
    df = pd.read_csv(path)
    all_data.append(df)

In [4]:
X = np.array(all_data)
X = np.transpose(X, (0, 2, 1))

In [5]:
df = pd.read_csv('/root/projects/CP/Projects/Signal/label_exp.csv')
y = np.array(df['label'])
y

array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1])

In [6]:
import torch   #general pytorch
import torch.nn as nn  #neural network module
import torch.nn.functional as F  #useful functions like softmax, or relu

#pip install torchvision; conda install torchvision
from torchvision import datasets, transforms  #transforms for image processing
from torch.utils.data import DataLoader, Dataset       #dataloader for preparing batch


import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [17]:
# Your code
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13, 16, 45000) (4, 16, 45000) (13,) (4,)


In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=999)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(11, 16, 45000) (2, 16, 45000) (11,) (2,)


In [19]:
from sklearn.preprocessing import MinMaxScaler

#scale transform each channel independently
scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = MinMaxScaler(feature_range=(-1, 1))
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i]) 

for i in range(X_val.shape[2]):
    X_val[:, :, i]   = scalers[i].transform(X_val[:, :, i])     
    
for i in range(X_test.shape[2]):
    X_test[:, :, i]  = scalers[i].transform(X_test[:, :, i]) 

In [20]:
print(X_train.mean(), X_val.mean(), X_test.mean())
print(X_train.min(), X_val.min(), X_test.min())
print(X_train.max(), X_val.max(), X_test.max())

0.17394497961134747 -0.34915787177665036 0.12544181839149665
-1.0 -5.276083248178158 -1.6923017702572953
1.0000000000000004 0.9985499015664524 1.848048482814936


In [21]:
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

# Cast data to dataloader for more convenience
training_set = TensorDataset(X_train_tensor, y_train_tensor)
testing_set = TensorDataset(X_test_tensor, y_test_tensor)
validation_set = TensorDataset(X_val_tensor, y_val_tensor)

train_batch_size = 5
val_batch_size = len(validation_set)
test_batch_size = len(testing_set)

train_loader = DataLoader(training_set, train_batch_size, shuffle=True)
val_loader = DataLoader(validation_set, val_batch_size, shuffle=True)
test_loader = DataLoader(testing_set, test_batch_size, shuffle=True)

In [22]:
fake_eeg_data = torch.randn((11, 50, 44998)) #(batch_size, channels, length)
maxpool1d = nn.MaxPool1d(2,2)
out = maxpool1d(fake_eeg_data)
out.shape

torch.Size([11, 50, 22499])

In [23]:
class eegConv1d(nn.Module):
    def __init__(self, input_size = 8, out_size=2):
        super().__init__()
        self.c1 = nn.Conv1d(input_size, 50, kernel_size = 3)
        self.c2 = nn.Conv1d(50, 30, kernel_size = 3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.maxpool1d = nn.MaxPool1d(2,2)
        self.linear = nn.Linear(30 * 22497, out_size) #taking the last hidden state
        
    def forward(self, seq):
        #convo layer 8 -> 50 -> 30
        #seq shape: (11, 50, 45000)
        out = self.c1(seq)
        out = self.relu(out)
        out = self.dropout(out)
        # out shape: (11, 50, 44998)
        out = self.maxpool1d(out)
        # out shape: (11, 50, 22499)
        out = self.c2(out)
        out = self.relu(out)
        # out shape: (11, 30, 22496)
        out = out.reshape(seq.size(0), -1)
        #out shape: (30, 30*22496)
        out = self.linear(out)
        #out shape: (30*22496, 2)
        return out

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
# torch.manual_seed(999999)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = eegConv1d(input_size=16).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [26]:
epochs = 20

best_valid_loss = float('inf')
model.train()

#print(f"Training {type(model).__name__}")

for i in range(epochs):
    train_total = 0
    train_correct = 0
    val_total   = 0  
    val_correct = 0
    train_acc   = 0
    val_acc     = 0
    
    for X_train, y_train in train_loader:
    
        start_time = time.time()
        
        X_train = X_train.float().to(device)
        y_train = y_train.type(torch.LongTensor).to(device)

        #print(X_train.shape, X_train.dtype)

        yhat_train = model(X_train)
        
        #train acc
        _, predicted = torch.max(yhat_train.data, 1)  #returns max value, indices
        train_total += y_train.size(0)  #keep track of total
        train_correct += (predicted == y_train).sum().item()  #.item() give the raw number
        train_acc = 100 * (train_correct / train_total)
        
        #print(y_train.shape, y_train.dtype)
        
        train_loss = criterion(yhat_train, y_train)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        end_time = time.time()

        #val accuracy
        for X_val, y_val in val_loader:
            X_val = X_val.float().to(device)
            y_val = y_val.type(torch.LongTensor).to(device)
            yhat_val  = model(X_val)
            val_loss     = criterion(yhat_val, y_val)
            _, predicted = torch.max(yhat_val.data, 1)  #returns max value, indices
            val_total += y_val.size(0)  #keep track of total
            val_correct += (predicted == y_val).sum().item()  #.item() give the raw number
            val_acc = 100 * (val_correct / val_total)

        #save the best model
        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            #print("Model:{} saved.".format(type(model).__name__))
            torch.save(model.state_dict(), '/root/projects/CP/Projects/Model/chornicconvo1d.pth.tar')
            best_model_index = i

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {i:2.0f} | Train acc: {train_acc: 2.2f} | " +
          f"loss: {train_loss:2.5f} | Val acc: {val_acc: 2.2f} | " +
          f"loss: {val_loss:2.5f} | Time: {epoch_mins}m {epoch_secs}s")

Epoch:  0 | Train acc:  72.73 | loss: 0.00000 | Val acc:  33.33 | loss: 573.93500 | Time: 0m 0s
Epoch:  1 | Train acc:  81.82 | loss: 4448.88477 | Val acc:  83.33 | loss: 0.00000 | Time: 0m 0s
Epoch:  2 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 1035.77136 | Time: 0m 0s
Epoch:  3 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 1409.53467 | Time: 0m 0s
Epoch:  4 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 1672.25049 | Time: 0m 0s
Epoch:  5 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 1895.83105 | Time: 0m 0s
Epoch:  6 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 2102.13818 | Time: 0m 0s
Epoch:  7 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 2263.89429 | Time: 0m 0s
Epoch:  8 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 2403.82666 | Time: 0m 0s
Epoch:  9 | Train acc:  100.00 | loss: 0.00000 | Val acc:  50.00 | loss: 2509.31812 | Time: 0m 0s
Epoch: 10 | Train acc: 

In [31]:
yhat = model(X_test_tensor.float().to(device))
#testloss = criterion(yhat, y_test_tensor.type(torch.LongTensor).to(device))
_, predicted = torch.max(yhat.data, 1)

predicted

tensor([1, 0, 0, 0], device='cuda:0')

In [32]:
y_test

array([1, 0, 1, 0])